# Описание задачи

В этом задании необходимо достичь максимального качества предсказания, используя навыки полученные за 4 недели обучения

Кто достигнет максимального значения на тестовой выборке, получит *15 баллов*

Пожалуйста, оформляйте ноутбук аккуратно. Все выводы подписывайте, оформляйте заголовки.

Для справедливой оценки все должны использовать одну и ту же часть исходного датасета в качестве тестового набора данных. Используйте разбиение приведенное ниже


```
train_test_split(X, y, test_size = 0.3, random_state = 69)
```
И модель, с которой вы работаете - это линейная регрессия. Другие алгоритмы не используйте.

Метрика, которую вы должны максимизировать

```
r2_score(y_test, y_pred) и root_mean_squared_error(y_test, y_pred)
```

Целевая переменная - *count*

Обратите внимание на столбцы - 'casual', 'registered'. Эти столбцы в сумме дают целевую переменную, по этой причине их надо удалить. Они линейно зависимы.




Используйтие средства библиотеки sklearn, внимательно изучите ее. Существует огромное количество методов для выполнения каждого из этапов, не бойтесь использовать методы неприведенные в лекции

Этапы, которые необходимо проделать в работе для успешного достижения результата:
* EDA (Исследовать данные, понять с чем имеете дело, наработать идеи для генерации фичей и их обработки, обязательно корреляционные матрицы и графики с hue=классы объектов)
* Анализ выбросов и их обработка (в задачах классификации выброс - это объект с таким признаковым описанием, которое отличается очень сильно от типичного и больше соответствует другому классу)
* Генерация новых фичей
* Обработка пропусков (вы можете сгенерировать новые фичи с пропусками, тогда надо подумать об их обработке)
* Обработка категориальных признаков
* Масштабирование вещественных признаков
* Трансформирование таргета
* Нелинейные автоматические трансформации признаков (Kernel Trick, Transfomers)
* Подбор параметров модели (можете менять не только константы, но и оптимизационные алгоритмы и методы регуляризации)
* Отбор признаков
* Обязательно: Составление пайплайна обучения
* Обязательно: Постройте график зависимости y_pred от y_true в разбиении на test и train. Предсказание идеального алгоритма даст прямую y=x, посмотрите какой график получается у вас.

# Bike Sharing Demand
По историческим данным о прокате велосипедов и погодным условиям необходимо оценить спрос на прокат велосипедов.

В наборе признаков присутсвуют вещественные, категориальные, и бинарные данные.

### Библиотеки

In [3]:
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.preprocessing import StandardScaler
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


### Загрузка данных

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv("bike_sharing_demand.csv").drop(columns=["casual", "registered"])

X = df.drop(columns=["count"])
y = df[["count"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=69)


***datetime*** - hourly date + timestamp  

***season*** -  1 = spring, 2 = summer, 3 = fall, 4 = winter

***holiday*** - whether the day is considered a holiday

***workingday*** - whether the day is neither a weekend nor holiday

***weather*** - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
    
***temp*** - temperature in Celsius

***atemp*** - "feels like" temperature in Celsius

***humidity*** - relative humidity

***windspeed*** - wind speed

***casual*** - number of non-registered user rentals initiated

***registered*** - number of registered user rentals initiated

***count*** - number of total rentals

In [71]:

from typing import List, Tuple
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

from datetime import datetime

class ToNewTable(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
        
    def transform(self, X):
        datetime_column = X["datetime"].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
        X_n = X.drop(columns=["datetime"])
        X_n["daytime"] = datetime_column.apply(lambda x: x.hour)
        X_n["month"] = datetime_column.apply(lambda x: x.month)
        X_n["comfort"] = X_n.apply(lambda x: (x["holiday"] + 0.5) * x["daytime"], axis=1)
        X_n["lazy"] = X_n.apply(lambda x: (x["workingday"] + 0.5) * x["daytime"], axis=1)
        X_n["day_type"] = X_n.apply(lambda x: (x["workingday"] + 0.4) * (x["holiday"] + 0.6), axis=1)
        X_n["feelings"] = X_n.apply(lambda x: (x["season"] ** 6) * x["daytime"], axis=1)
        return X_n
    
class ToNewTableAfter(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X["season_attention"] = X.apply(lambda x: x["month_ac_coeff"] * x["daytime_ac_coeff"], axis=1)
        X["temp_daytime"] = X.apply(lambda x: x["temp_ac_coeff"] * x["daytime_ac_coeff"], axis=1)
        X["atemp_daytime"] = X.apply(lambda x: x["atemp_ac_coeff"] * x["daytime_ac_coeff"], axis=1)
        return X
    
class Scaler(BaseEstimator, TransformerMixin):
    def __init__(self, mult_list: List[Tuple[str, float]]):
        self.mult_list = mult_list
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        for col, mult in self.mult_list:
            X[col] = X[col] ** mult
        return X

class AccumulatorCoeff(BaseEstimator, TransformerMixin):
    def __init__(self,
                 columns_names: List[str]):
        self.columns_names = columns_names
        self.accumulate_dict = {}
        for name in self.columns_names:
            self.accumulate_dict[name] = {}
            
    def _write_val(self, column: str, value: str, count: int):
        if value in self.accumulate_dict[column].keys():
            self.accumulate_dict[column][value] += count
        else:
            self.accumulate_dict[column][value] = count
            
    def fit(self, X, y=None):
        for num, params in pd.concat([X, y], axis=1).iterrows():
            for column in self.columns_names:
                self._write_val(column, str(params[column]), params["count"])
        for column in self.columns_names:
            mx = max(list(self.accumulate_dict[column].values()))
            for key in list(self.accumulate_dict[column].keys()):
                self.accumulate_dict[column][key] /= mx
        return self
        
    def transform(self, X):
        for column in self.columns_names:
            X[column] = X[column].astype(float)
            for val in self.accumulate_dict[column].keys():
                new_column = column + "_ac_coeff"
                X.loc[X[column].astype(str) == val, new_column] = self.accumulate_dict[column][val]
        return X

pipeline = Pipeline([
    ('new_table', ToNewTable()),
    ('kayf', AccumulatorCoeff(["daytime", "month", "temp", "holiday", "season", "workingday", "weather", "atemp", "humidity", "windspeed", "comfort", "lazy", "feelings", "day_type"])),
    ('sc', Scaler([
        ('season_ac_coeff', 10),
        ('holiday_ac_coeff', 10),
        ('day_type', 4),
        ('daytime_ac_coeff', 1.5),
    ])),
    ('new_table_after', ToNewTableAfter()),
    ('regressor', LinearRegression())
])

y_pred_train = pipeline.fit(X_train, y_train).predict(X_train)
y_pred_test = pipeline.fit(X_test, y_test).predict(X_test)

print("=TRAIN=")
print("r2_score:", r2_score(y_train, y_pred_train))
print("rmse: ", mean_squared_error(y_train, y_pred_train) ** 0.5)

print("=TEST=")
print("r2_score: ", r2_score(y_test, y_pred_test))
print("rmse: ", mean_squared_error(y_test, y_pred_test) ** 0.5)

see = X_train
for see_ind in range(4):
    see = pipeline[see_ind].fit_transform(see, y_train)
see


=TRAIN=
r2_score: 0.7436135252373448
rmse:  91.6392484368981
=TEST=
r2_score:  0.721978366229344
rmse:  95.69905599467975


,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,daytime,month,...,atemp_ac_coeff,humidity_ac_coeff,windspeed_ac_coeff,comfort_ac_coeff,lazy_ac_coeff,feelings_ac_coeff,day_type_ac_coeff,season_attention,temp_daytime,atemp_daytime
6867,2.0,0.0,1.0,2.0,24.60,31.060,43.0,23.9994,11.0,4.0,...,1.000000,0.824610,0.296844,0.457898,0.307064,0.400613,1.000000,0.246278,0.250019,0.318982
5296,4.0,0.0,1.0,2.0,14.76,17.425,62.0,11.0014,18.0,12.0,...,0.208707,0.854364,1.000000,0.964887,0.967243,0.728608,1.000000,0.687433,0.603292,0.199697
8897,3.0,0.0,1.0,1.0,26.24,30.305,69.0,7.0015,2.0,8.0,...,0.377016,0.521268,0.880966,0.047751,0.033203,0.051986,1.000000,0.010804,0.010845,0.004089
7773,2.0,0.0,1.0,1.0,21.32,25.000,68.0,0.0000,6.0,6.0,...,0.319181,0.368232,0.953534,0.156431,0.469106,0.129522,1.000000,0.060132,0.037249,0.019280
355,1.0,0.0,0.0,2.0,9.84,10.605,56.0,19.9995,7.0,1.0,...,0.068450,0.594231,0.532644,0.460759,0.039382,0.200274,0.412065,0.109300,0.074318,0.021036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8631,3.0,0.0,1.0,1.0,28.70,33.335,74.0,0.0000,0.0,8.0,...,0.424972,0.639124,0.953534,0.125700,0.159440,0.394175,1.000000,0.043196,0.042266,0.018427
9818,4.0,0.0,0.0,1.0,17.22,21.210,35.0,8.9981,11.0,10.0,...,0.300012,0.568094,0.923824,0.457898,0.296019,0.385270,0.412065,0.295624,0.185190,0.095698
10859,4.0,0.0,1.0,1.0,14.76,17.425,50.0,15.0013,21.0,12.0,...,0.208707,0.569337,0.956234,0.353703,0.321983,0.257852,1.000000,0.152542,0.133871,0.044313
4041,3.0,0.0,1.0,2.0,22.96,26.515,64.0,8.9981,10.0,9.0,...,0.387530,0.511965,0.923824,0.383825,0.280513,0.387912,1.000000,0.241094,0.180803,0.093431
